In [1]:
# For extracting files from Amazon S3 Buckets
import boto3
from botocore import UNSIGNED
from botocore.client import Config

# For PySpark
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType

# For Delta Lake
from delta import *
#import cloudstorage as gcs

# For VGG16 Model
from keras.applications.vgg16 import VGG16 
from keras.models import Model
from keras.applications.vgg16 import preprocess_input

# For PCA and Kmeans
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Working with images
from PIL import Image
import io
import base64
import numpy as np

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-k_8ygc4y because the default path (/home/developer/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
# Connect to Amazon S3 Buckets
s3 = boto3.resource('s3', config=Config(signature_version=UNSIGNED))
bucket = s3.Bucket("airborne-obj-detection-challenge-training")
bucket_objects = bucket.objects.filter(Prefix="part1/Images/1497343b9d90411db5c305e785be9032")

/usr/local/lib/python3.6/dist-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [3]:
import os
import sys
os.environ["SPARK_HOME"] = "/opt/spark-3.0.1-bin-hadoop2.7"
os.environ["PYTHONPATH"] = "$SPARK_HOME/python:$SPARK_HOME/python/lib/py4j-0.10.9-src.zip"
os.environ["PATH"] = "$SPARK_HOME/bin:$SPARK_HOME/python"
#os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[2] pyspark-shell"
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#os.environ["PATH"] = "$SPARK_HOME/bin:$SPARK_HOME/python:$JAVA_HOME/bin:$PYSPARK_SUBMIT_ARGS/bin"

In [4]:
pyspark_submit_args = os.environ.get("PYSPARK_SUBMIT_ARGS", "")
if not "pyspark-shell" in pyspark_submit_args: pyspark_submit_args += " pyspark-shell"
os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

In [8]:
from pyspark import SparkConf, SparkContext, SQLContext

conf = (SparkConf()
         .setAppName("S3 Configuration Test")
         .set("spark.executor.instances", "1")
         .set("spark.executor.cores", 1)
         .set("spark.executor.memory", "2g"))
sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)

RuntimeError: Java gateway process exited before sending its port number

In [7]:
builder = SparkSession.builder.appName("drones") \
    .config("spark.executor.memory", "15g") \
    .config("spark.driver.memory", "15g") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.1") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()
sc = spark.sparkContext
spark

RuntimeError: Java gateway process exited before sending its port number